In [1]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
from VideoSwinTransformer import *
import os
from collections import OrderedDict
import sys

c:\Python\Python396\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']
# download_weight_command = f"wget {link} -O {name}.pth"
# os.system(download_weight_command)

# model = SwinTransformer3D(**cfg)

In [3]:
def get_model(num_classes,cfg, shape_of_input=(10,3,32,224,224)):
    inputs = tf.keras.Input(shape_of_input[1:])
    backbone = SwinTransformer3D(**cfg, shape_of_input=shape_of_input)
    x = backbone(inputs, training= False)
    outputs = I3DHead_tf(num_classes, 768, training=True)(x)
    return tf.keras.Model(inputs, outputs)

In [4]:
shape_of_input = (5, 3, 8, 112,112)
model = get_model(5, cfg, shape_of_input=shape_of_input)

X = tf.random.normal(shape_of_input)
y = tf.random.uniform((shape_of_input[0],1), 0, 5, tf.dtypes.int32)
model.summary()

shape_of_input:  (5, 3, 8, 112, 112)
backbone (None, 768, 4, 4, 4)
5 768 {'type': 'CrossEntropyLoss'} avg 0.5 0.01 {'training': True}
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 8, 112, 112)]  0         
                                                                 
 swin_transformer3d (SwinTra  (None, 768, 4, 4, 4)     29694438  
 nsformer3D)                                                     
                                                                 
 i3d_head_tf (I3DHead_tf)    (None, 5)                 3845      
                                                                 
Total params: 29,698,283
Trainable params: 27,854,315
Non-trainable params: 1,843,968
_________________________________________________________________


In [5]:
# inputs = tf.keras.Input(shape_of_input[1:])
# model(inputs, training=True)

In [6]:
y1 = model.predict(X)
y1

1/1 [==============================] - 6s 6s/step


array([[0.18136032, 0.19400212, 0.2036266 , 0.223305  , 0.19770586],
       [0.16859475, 0.20558141, 0.2069594 , 0.22127883, 0.19758564],
       [0.19096507, 0.19588126, 0.20332469, 0.2164006 , 0.19342834],
       [0.16823882, 0.19639531, 0.20868362, 0.20981662, 0.21686564],
       [0.18486339, 0.21553005, 0.1992743 , 0.19900191, 0.20133035]],
      dtype=float32)

In [7]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.SGD(), metrics=["accuracy"])

model.fit(X, y, epochs=2)

Epoch 1/2
1/1 [==============================] - 19s 19s/step - loss: 1.6722 - accuracy: 0.2000
Epoch 2/2
1/1 [==============================] - 4s 4s/step - loss: 1.2308 - accuracy: 0.6000


In [8]:
y1 = model.predict(X)
y1

1/1 [==============================] - 6s 6s/step


array([[0.185227  , 0.13570127, 0.03920101, 0.5234397 , 0.11643103],
       [0.19315024, 0.13914372, 0.03876765, 0.51896596, 0.1099724 ],
       [0.18852091, 0.12692855, 0.03683257, 0.54710597, 0.10061203],
       [0.17555618, 0.15028082, 0.03915363, 0.5172103 , 0.11779898],
       [0.18544194, 0.13939264, 0.03600707, 0.5323337 , 0.10682465]],
      dtype=float32)

In [9]:
y2 = model(X, training=False)
y2

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0.18522704, 0.13570127, 0.03920102, 0.5234397 , 0.11643103],
       [0.19315022, 0.13914372, 0.03876765, 0.51896596, 0.10997241],
       [0.18852088, 0.12692855, 0.03683258, 0.54710597, 0.10061203],
       [0.17555618, 0.15028082, 0.03915364, 0.51721036, 0.11779898],
       [0.18544196, 0.13939266, 0.03600707, 0.5323337 , 0.10682464]],
      dtype=float32)>

In [10]:
model.layers[1].layers[5].weights

[<tf.Variable 'swin_transformer3d/basic_layer_3/swin_transformer_block3d_10/layer_normalization_23/gamma:0' shape=(768,) dtype=float32, numpy=
 array([0.9999943 , 0.9999994 , 0.9999945 , 0.9999876 , 0.99998677,
        0.9999591 , 1.0000187 , 0.9999977 , 0.99999017, 0.99998385,
        0.9999919 , 1.000028  , 1.0000297 , 0.9999697 , 0.99998075,
        1.0000004 , 0.99998987, 1.0000099 , 0.99999654, 0.99998665,
        1.00002   , 0.999936  , 0.99998695, 0.9999758 , 1.000008  ,
        0.99994165, 0.9999836 , 0.9999945 , 1.0000311 , 0.9999801 ,
        1.000005  , 1.0000148 , 1.0000211 , 1.0000212 , 0.9999206 ,
        1.0000051 , 0.9999806 , 1.0000267 , 1.0000222 , 1.0000148 ,
        1.0000023 , 1.000001  , 0.9999937 , 1.0000141 , 1.0000055 ,
        1.0000238 , 1.0000006 , 0.9999878 , 0.99995637, 0.99999756,
        1.0000224 , 0.9999832 , 0.99998957, 1.0000128 , 0.99997395,
        0.9999752 , 1.0000136 , 0.9999871 , 1.0000174 , 1.0000637 ,
        1.0000032 , 0.9999932 , 0.9999958